In [ ]:
!pip install seaborn imutils --quiet

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import math, random
import imutils
import os
import gc
import cv2
from tqdm import tqdm

In [ ]:
show_visuals = True
test_on_data = True

In [ ]:
train_csv_path = "/kaggle/input/rsna-breast-cancer-detection/train.csv"
train_image_dir = "/kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images"

In [ ]:
train = pd.read_csv(train_csv_path)
train

In [ ]:
images_ = train[['patient_id', 'image_id', 'cancer','difficult_negative_case']]
images_['file_path'] = images_.apply(lambda x: os.path.join(train_image_dir, str(x.patient_id), str(x.image_id) + '.png'), axis=1)
images_.head(1)

#### The number and proportion of the positive and negative cases

In [ ]:
pro = images_.groupby("cancer")['difficult_negative_case'].value_counts().reset_index()
pro['proportion'] = pro['count'] / len(images_)
display(pro)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def has_cancer(c):
    return c > 0

cancer_per_patient = train.groupby("patient_id")["cancer"].sum().apply(has_cancer)

if show_visuals:
    ax = sns.countplot(x=cancer_per_patient)
    ax.bar_label(ax.containers[0])

    plt.xlabel("Cancer")
    plt.ylabel("Count")
    plt.title("Number of patients with cancer")

    plt.show()

#### The age distribution

In [ ]:
# g = sns.FacetGrid(train, col = 'cancer')
# g.map(sns.histplot, 'age' )

## Alternative: detailed and bigger view
if show_visuals:
    sns.displot(train[train['cancer']==1].age, bins=50, kde=False)
    sns.displot(train[(train['cancer']==0) & (train['difficult_negative_case']==True)].age, bins=50, kde=False)
    sns.displot(train[(train['cancer']==0) & (train['difficult_negative_case']==False)].age, bins=50, kde=False)
    plt.show()

#### As Cancer and not cancer are unbalanced interms of samples , we would have to sample data from both class so that model gets the general undrstanding of what both type of data looks like

In [ ]:
# Keep only certain negatives
not_cancer = images_[(images_['cancer'] == 0) & (images_['difficult_negative_case'] == False)]
# And all cancers
with_cancer = images_[images_['cancer'] == 1]

In [ ]:
def crop_image(img, th=0):
    mask = img > th
    return img[np.ix_(mask.any(1),mask.any(0))]

# Doubles the image's width and height by adding black margins
def add_margins(img):
    (h, w) = img.shape[:2]

    # Using the diagonal is a correct way to do that
    mx = math.ceil(2 * math.sqrt(h*h + w*w))
    # A new square image will contain any version of the original image rotated around the center
    new_img = np.zeros((mx,mx), np.uint8)
    
    hh = mx // 2 - h
    wh = mx // 2
    
    # Position the original image with its bottom left corner on the center
    new_img[hh:hh+h, wh:wh+w] = img
    return new_img

In [ ]:
orb = cv2.ORB_create()

In [ ]:
# This method rotates the image, gets the keypoints and then transforms them back to the original images's coordinates
def rotate_and_reverse_keypoints(img, angle):
    # Get the matrix for rotation around the center of the image
    height, width = img.shape[:2] 
    center = (width/2, height/2) 
    cos = math.cos(math.radians(-angle));
    sin = math.sin(math.radians(-angle));
    
    img_new = imutils.rotate(img, angle = angle)
    
    # Detect ORB keypoints
    kp = orb.detect(img_new, None)

    # Consider that y axis is directed down in screen coordinates
    coords_transformed = [(k.pt[0] - center[0], center[1] - k.pt[1]) for k in kp]
    # Rotate all the points around the center of coordinates
    coords_transformed = [(cos * k[0] - sin * k[1], sin * k[0] + cos * k[1]) for k in coords_transformed]
    
    # Translate all the points back and add to the combined list
    coords_final = [(int(k[0] + center[0]), int(center[1] - k[1])) for k in coords_transformed]
    
    return coords_final

In [ ]:
def draw_points(img, points, radius=5):
    color = (255, 0, 0)
    thickness = 1
    
    # This is to convert the image from grayscale to RGB
    img_new = np.stack((img,)*3, axis=-1)
    
    for coord in points:
        #Draw circles for given points 
        point = (int(coord[0]), int(coord[1]))
        img_new = cv2.circle(img_new, point, radius, color, thickness)
    
    return img_new

In [ ]:
# Converts the coordinates as if margins removed
# Removes the black margins added for rotation
def remove_margins_shift_coords(points, img, width, height):
    # Get the size of the image and calculate the center coordinates
    (h, w) = img.shape[:2]
    center = (w//2, h//2)
    new_points = [(int(p[0])-center[0], int(p[1])-center[1]+height) for p in points]
    new_img = img[center[1]-height:center[1], center[0]:center[0]+width]
    return new_img,new_points

In [ ]:
# Returns the number of keypoints for the 2D histogram
def count_points(points, img, width, height):
    # Get the size of the image
    (h, w) = img.shape[:2]
    rows, cols = (math.ceil(h/height), math.ceil(w/width))
    
    # Introduce a 2D list for storing numbers
    number_points = np.zeros((rows+1,cols+1),dtype = np.uint8)
    for coord in points:
        number_points[math.floor(coord[1]/height)][math.floor(coord[0]/width)] += 1
        
    return number_points

In [ ]:
# Draw a 2D histogram on a given image using the numbers
def draw_histogram_old(img, points, width, height, colormin=(0, 255, 255), colormax=(255, 0, 0)):
    # Count density of points in a given raster of rectangular cells
    numbers = count_points(points, img, width, height)
    
    # If the image is a grayscale one, turn it to RGB
    if len(img.shape) < 3:
        img_new = np.stack((img,)*3, axis=-1)
    else:
        img_new = np.array(img)
    
    # Find the maximum and minimum in the 2D list
    num_max = max(map(max, numbers))
    num_min = min(map(min, numbers))
    # Loop through all rows and columns
    (rows, cols) = numbers.shape[:2]
    (img_height, img_width) = img.shape[:2]
    # Generate a histogram mask image
    img_histo = np.zeros_like(img_new)
    for i in range(rows):
        for j in range(cols):
            n = (numbers[i,j] - num_min)/num_max
            # Interpolate color between the colormin and colormax values
            color = ((1-n)*colormin[0] + n*colormax[0], (1-n)*colormin[1] + n*colormax[1], (1-n)*colormin[2] + n*colormax[2])
            # Strangely it does not draw if the coordinates are outside the image area
            img_histo = cv2.rectangle(img_histo, (j*width, i*height), (min(width*(j+1)-1, img_width-1), min(height*(i+1)-1, img_height-1)), color, -1)

    #img_new = cv.bitwise_and(img_new, img_histo) # Bitwise does not work as intended
    # Use the grayscale image as intensity for the RGM image
    for i in range(img_height):
        for j in range(img_width):
            # Overflow happens if you multiply first instead of dividing to 255
            img_new[i, j][0] = round(img_histo[i, j][0] / 255 * img_new[i, j][0])
            img_new[i, j][1] = round(img_histo[i, j][1] / 255 * img_new[i, j][1])
            img_new[i, j][2] = round(img_histo[i, j][2] / 255 * img_new[i, j][2])

    #return img_histo
    return img_new

In [ ]:
# Draw a 2D histogram on a given image using the numbers
def draw_histogram(img, number_points):
    # Scaling the density to [0,255]
    min_val = np.min(number_points)
    max_val = np.max(number_points)
    number_points = (255.0*(number_points - min_val) / (max_val - min_val)).astype(int)
        
    (rows, cols) = number_points.shape[:2]
    (img_height, img_width) = img.shape[:2]
    
    # Stretching the keypoint density array to the image's size
    number_points = np.repeat(np.repeat(number_points,(img_width//cols), axis=0), (img_height//rows), axis=1)
    
    # If the image is a grayscale one, turn it to RGB
    if len(img.shape) < 3:
        img_new = np.stack((img,)*3, axis=-1)
    else:
        img_new = np.array(img)
    
    min_h = min(img_height,number_points.shape[0])
    min_w = min(img_width,number_points.shape[1])
    
    # the slicing is done to align the size of matrices
    img_new[:min_h,:min_w,0] = number_points[:min_h,:min_w]
    img_new[:min_h,:min_w,1] = number_points[:min_h,:min_w]
    return img_new

In [ ]:
def get_subimage(img, frame_idx,frame_width,frame_height):
    (img_height, img_width) = img.shape[:2]
    y_s = frame_idx[0]*frame_height
    y_e = min(y_s + frame_height,img_height) # not to go beyond the image size
    x_s = frame_idx[1]*frame_width
    x_e = min(x_s + frame_width,img_width) # not to go beyond the image size
    
    return img[y_s:y_e,x_s:x_e]

# Returns array of images of given size with highest numbers in histogram
def get_images(img, number_points, width, height, max_images, black_threshold = 0.55):
    (rows, cols) = number_points.shape[:2]
    (img_height, img_width) = img.shape[:2]
    
    # Sort the array values (to have it descending, multiplied by -1 )
    arr_srt = np.argsort(-1*number_points,axis=None)
    # The result is flattened (due to axis = None)
    unidx = np.unravel_index(arr_srt,number_points.shape)
    # Make the result as tuples - coordinates of the areas with max density
    idx_tpls = list(zip(unidx[0],unidx[1]))
    
    i = 0
    cnt = 0
    img_arr = np.zeros((0,width,height))
    # This will keep the index of the skipped images (due to size or black pixels)
    skipped_images = []
    
    # Keep taking top max_images from the ordered list 
    while i < len(idx_tpls) and (cnt < max_images):
        tmp = get_subimage(img, idx_tpls[i], width, height)
    
        # Count if the black pixels are not in majority and the shapes correspond
        if (np.count_nonzero(tmp<10) < (width*height*black_threshold)) and (tmp.shape == img_arr.shape[1:3]):
            img_arr = np.append(img_arr,np.expand_dims(tmp, axis=0),axis=0)
            cnt += 1
        else:
            skipped_images.append(i)
        i += 1

    # If the matrix is not full, then add smaller images by padding them
    if img_arr.shape[0] != max_images:
        required_imgs = max_images - img_arr.shape[0]
        i = 0
        while i < required_imgs:
            tmp = get_subimage(img, idx_tpls[i],width,height)
            (h,w) = tmp.shape
            tmp = np.pad(tmp, ((0,height-h),(0,width-w)), 'constant')
            img_arr = np.append(img_arr,np.expand_dims(tmp, axis=0),axis=0)
            i += 1
    
    return img_arr

In [ ]:
# Experiments with the Konul x.
# 312 /kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/26664/1409401831.png
# 253 /kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/23224/1242778806.png
# 80 /kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/13331/1949261883.png
# 252 /kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/23101/732919056.png
# 117 /kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/15945/890480233.png

In [ ]:
sample_id = 252 #random.randint(0,400)
#fname = with_cancer['file_path'].iloc[sample_id]
#fname = '/kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/23101/732919056.png'
fname = '/kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/13331/1949261883.png'
fname = '/kaggle/input/oncologyclinicdata/LN00030001.png'
#82 /kaggle/input/rsna-mammogram-pngs-enhanced/png_train_images/1336/1475901500.png
print(sample_id,fname)
img_init = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
img_init = cv2.resize(img_init, (300,500)) # for the big images 

img_cropped = crop_image(img_init, 2)
img_margin = add_margins(img_cropped)

# Remember the size of the initial cropped image
(height_cropped, width_cropped) = img_cropped.shape[:2]

In [ ]:
coords_combined = []

for angle in [15,-15,30,-30,45]:
    coords = rotate_and_reverse_keypoints(img_margin,30)
    coords_combined += coords

img_rm_margins, coords_combined = remove_margins_shift_coords(coords_combined, img_margin, width_cropped, height_cropped)

In [ ]:
frame_width = 50
histogram_numbers = count_points(coords_combined, img_rm_margins, frame_width, frame_width)

In [ ]:
if show_visuals:
    fig, axarr = plt.subplots(1,5, figsize=(15, 6))
    fig.suptitle('Keypoint detection')
    # axarr[0].set_title('Original image', loc='right')
    # axarr[0].imshow(img_init, cmap='gray')
    axarr[0].set_title('Cropped image', loc='right')
    axarr[0].imshow(img_cropped, cmap='gray')
    # axarr[2].set_title('Image with margins', loc='right')
    # axarr[2].imshow(img_margin, cmap='gray')
    axarr[1].set_title('All keypoints combined', loc='right')
    axarr[1].imshow(draw_points(img_rm_margins, coords_combined, 2), cmap='gray')
    axarr[2].set_title('Density of the keypoints', loc='right')
    axarr[2].imshow(histogram_numbers)
    axarr[3].set_title('High density regions', loc='right')
    axarr[3].imshow(draw_histogram_old(img_rm_margins, coords_combined, frame_width, frame_width))
    axarr[4].set_title('High density regions', loc='right')
    axarr[4].imshow(draw_histogram(img_rm_margins, histogram_numbers))

    plt.show()

In [ ]:
# Showing the X common keypoints and the 20x20 regions around them
if show_visuals:
    frame_cnt = 15

    a = 3
    b = frame_cnt // a

    fig, axarr = plt.subplots(b, a, constrained_layout = True, figsize=(6, 10))
    fig.suptitle('20 common keypoints and the 20x20 regions around them')
    img_arr  = get_images(img_rm_margins, histogram_numbers, frame_width, frame_width, frame_cnt, black_threshold = 0.8)

    for i in range(b):
        for j in range(a):
            axarr[i, j].set_xticklabels([])
            axarr[i, j].set_yticklabels([])
            axarr[i, j].imshow(img_arr[i*a+j,:,:], cmap='gray')
    plt.show()

In [ ]:
# Combining all in one function: receives an image and returns N frames from the interest areas
def get_interest_frames(img, frame_width = 20, frame_cnt = 20):
    img_cropped = crop_image(img, 2)
    img_margin = add_margins(img_cropped)
    
    (height_cropped, width_cropped) = img_cropped.shape[:2]
    
    coords_combined = []

    for angle in [15,-15,30,-30,45]:
        coords = rotate_and_reverse_keypoints(img_margin,30)
        coords_combined += coords

    img_rm_margins, coords_combined = remove_margins_shift_coords(coords_combined, img_margin, width_cropped, height_cropped)
    
    histogram_numbers = count_points(coords_combined, img_rm_margins, frame_width, frame_width)
    img_arr  = get_images(img_rm_margins, histogram_numbers, frame_width, frame_width, frame_cnt)
    
    return img_arr

In [ ]:
# This part is used to run the trained model on the images of Oncological Clinic
import tensorflow as tf
import keras.api._v2.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import activations

class BC3DConvNet(tf.keras.Model):
    def __init__(self, frame_cnt, frame_width):
        super(BC3DConvNet, self).__init__()

        self.conv1 = Conv2D(filters=frame_cnt*16, kernel_size=(3, 3), input_shape=(frame_width, frame_width, frame_cnt))
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2D(filters=frame_cnt*16, kernel_size=(3, 3))
        self.bn2 = BatchNormalization()
        self.conv3 = Conv2D(filters=frame_cnt*32, kernel_size=(3, 3))
        self.bn3 = BatchNormalization()
        self.conv4 = Conv2D(filters=frame_cnt*64, kernel_size=(3, 3))
        self.bn4 = BatchNormalization()

        # Calculate the convolution's output size:
        tmp = np.zeros((1, frame_width, frame_width, frame_cnt))
        x = self.apply_conv(tmp)
        x = Flatten()(x)
        linear_input = x.shape[1]
        # ------------------------------------------------------

        self.fc1 = Dense(512, activation='relu')
        self.bn5 = BatchNormalization()
        self.fc2 = Dense(1024, activation='relu')
        self.bn6 = BatchNormalization()
        self.fc3 = Dense(512, activation='relu')
        self.bn7 = BatchNormalization()
        self.fc4 = Dense(2, activation='softmax')

    def apply_conv(self, x):
        x = activations.relu(self.conv1(x))
        x = self.bn1(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = activations.relu(self.conv2(x))
        x = self.bn2(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = activations.relu(self.conv3(x))
        x = self.bn3(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        x = activations.relu(self.conv4(x))
        x = self.bn4(x)
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        return x

    def call(self, x):
        conv_out = self.apply_conv(x)
        x = Flatten()(conv_out)
        x = self.fc1(x)
        x = self.bn5(x)
        x = Dropout(0.4)(x)
        x = self.fc2(x)
        x = self.bn6(x)
        x = Dropout(0.4)(x)
        x = self.fc3(x)
        x = self.bn7(x)
        x = Dropout(0.4)(x)
        x = self.fc4(x)
        return x

In [ ]:
frame_width = 50
frame_cnt = 15

if test_on_data:
    net = BC3DConvNet(frame_cnt, frame_width)
    net.compile(loss='binary_crossentropy', 
                optimizer=keras.optimizers.Adam(learning_rate=1e-3,weight_decay=0.1,clipnorm=1.0), 
                steps_per_execution=32,
                metrics=['accuracy'])
    onc_data = '/kaggle/input/oncologyclinicdata'
    net.load_weights('/kaggle/input/bc-keras-model/BC3DConvNet')
    for folder, subs, files in os.walk(onc_data):
        for filename in files:
            file_path = os.path.abspath(os.path.join(folder, filename))
            print(file_path)
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            # the images are bigger than the trianing images
            image = cv2.resize(image, (300,500))
            image = get_interest_frames(image, frame_width, frame_cnt)
            result = net.predict(tf.reshape(image,[1,frame_width,frame_width,frame_cnt]))
            prediction = 'POSITIVE' if (np.argmax(result)==0) else 'Negative'
            print("Result:",result, prediction,"Value:",result[0,np.argmax(result)])
            print()

In [ ]:
print(with_cancer.head())

In [ ]:
print(with_cancer.shape[0], not_cancer.shape[0])

In [ ]:
# Example is taken from here: 
# https://stackoverflow.com/questions/56418865/writing-and-reading-lists-to-tfrecord-example
import tensorflow as tf
from tensorflow.train import FloatList, BytesList, Int64List, Feature, Features, FeatureList, FeatureLists, Example

def _float_feature(value):
    return Feature(float_list=FloatList(value=value))

def _int_feature(value):
    return Feature(int64_list=Int64List(value=value))

def make_example(image, label):
    feat_arr2d = _float_feature(image.flatten())

    label_feature = _int_feature(label.flatten())
            
    example = Example(features=Features(feature={'image':feat_arr2d, 'label': label_feature}))
        
    return example.SerializeToString()

In [ ]:
def save_tfrecord(folder,image_id, file_path, add_info, label):
    try:
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if (label == 0):
            if random.random() > 0.5:
                image = cv2.flip(image, 1)
            else:
                # randomly rotate the image in [-25,25] rangle
                image = imutils.rotate(image, random.randint(0, 50)-25)
        image = get_interest_frames(image, frame_width, frame_cnt)
        
        fname = folder + '/' + str(image_id)+'_'+str(add_info)+'.tfrecord'
        with tf.io.TFRecordWriter(fname) as writer:
            writer.write(make_example(image, np.eye(2)[label].astype(int)))
    except Exception as e:
        print(f'Error loading image from {file_path}: {str(e)}')

In [ ]:
frame_cnt = 15
frame_width = 50

# Folder for positive samples
tf.io.gfile.mkdir('pos')
# Folder for negativetive samples
tf.io.gfile.mkdir('neg')

print('saving positives as TF records...')
for i in range(10):
    for idx,rec in tqdm(with_cancer.iterrows(), total=with_cancer.shape[0]):
        save_tfrecord('pos',rec['image_id'],rec['file_path'],i,0)

print('saving negetives as TF records...')
for idx,rec in tqdm(not_cancer.iterrows(), total=not_cancer.shape[0]):
    save_tfrecord('neg',rec['image_id'],rec['file_path'],0,1)

print('Done')

In [ ]:
def visualize_images(img_arr,frame_cnt):
    a = 5
    b = frame_cnt // a

    fig, axarr = plt.subplots(b, a, constrained_layout = True, figsize=(10, 5))

    for i in range(b):
        for j in range(a):
            axarr[i, j].set_xticklabels([])
            axarr[i, j].set_yticklabels([])
            axarr[i, j].imshow(img_arr[i*a+j,:,:], cmap='gray')
    plt.show()

In [ ]:
# Validation - let's check if we recorded images and labels correctly.

# Read from file and show
filenames = ['/kaggle/working/pos/846765464_0.tfrecord']
raw_dataset = tf.data.TFRecordDataset(filenames)

# A description of the features.
features_dict = {
    'image': tf.io.FixedLenFeature([frame_width*frame_width*frame_cnt], tf.float32),
    'label': tf.io.FixedLenFeature([2], tf.int64),
  }

def _parse_function(example):
    sample_record = tf.io.parse_single_example(example,features_dict)
    feats = sample_record['image']
    label = sample_record['label']
    
    feats = tf.reshape(feats,[frame_cnt,frame_width,frame_width])
    label = tf.reshape(label,[2,1])

    return feats,label

parsed_dataset = raw_dataset.map(_parse_function)
for img, label in parsed_dataset:
    print(label)
    visualize_images(img,frame_cnt)